In [13]:
import matplotlib.pyplot as plt
import skimage.util as util
import skimage.io as io
import numpy as np
import os as os 
import skimage.color as cl
import skimage.filters as filt 
import skimage.morphology as morph
import skimage.segmentation as seg
import skimage.measure as measure
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
import statistics as s
import skimage.exposure as exp

# Image Segmentation and Region Props

In [14]:
plt.rcParams['image.cmap'] = 'gray'

def segment_image(image):
    
    image = cl.rgb2gray(image.copy())
    image = filt.unsharp_mask(image, radius=1, amount=4)
    image = exp.equalize_hist(image,nbins=256)
    image = morph.opening(image,selem=morph.square(3))
    
    thresh = filt.threshold_otsu(image)
    segmented_image = image > thresh
    image_size = segmented_image.shape[0]* segmented_image.shape[1]
    
    segment_image = morph.binary_opening(segmented_image,selem = morph.square(10))
    segmented_image = morph.remove_small_objects(segmented_image,min_size=image_size/256)
    segmented_image = morph.remove_small_holes(segmented_image,min_size=64)
    
#     superimposed_image = seg.mark_boundaries(image,segmented_image,color=(1,0,0),outline_color=(1,0,0)) 
    
#     plt.figure()
#     plt.imshow(superimposed_image)
#     plt.show()
    
    return segmented_image

def segment_image_dynamic(image):
    
    image = cl.rgb2gray(image.copy())
    image = filt.unsharp_mask(image, radius=1, amount=4)
    image = exp.equalize_hist(image,nbins=256)
    image = morph.opening(image,selem=morph.square(3))
    
    seeds = np.zeros_like(image,dtype=np.uint)
    thresh = filt.threshold_otsu(image)
    seeds[image>thresh]=1
    seeds[image<=0.2]=2
    labels = seg.random_walker(image,seeds,beta=10)
    
    
    segmented_image = np.zeros((image.shape[0],image.shape[1]),dtype=bool)
    for i in range(0,labels.shape[0]):
        for j in range(0,labels.shape[1]):
            if(labels[i][j]==1):
                segmented_image[i][j]=True

    image_size = segmented_image.shape[0]* segmented_image.shape[1]
    
    segmented_image = morph.binary_opening(segmented_image,selem = morph.square(5))
    segmented_image = morph.remove_small_objects(segmented_image,min_size=image_size/256)
    segmented_image = morph.remove_small_holes(segmented_image,min_size=64)
    
    
    superimposed_image = seg.mark_boundaries(image,segmented_image,color=(1,0,0),outline_color=(1,0,0)) 
    
#     plt.figure()
#     plt.imshow(superimposed_image)
#     plt.show()
    
    return segmented_image

def calculate_region_props(labeled_image):
    area=[]
    eccentricity = []
    orientation = []
    bbox_area = []
    convex_area = []
    solidity = []
    extent = []
    
    region_props = measure.regionprops(labeled_image)

    for R in region_props:
        area.append(R['area'])
        eccentricity.append(R['eccentricity'])
        orientation.append(R['orientation'])
        bbox_area.append(R['bbox_area'])
        convex_area.append(R['convex_area'])
        solidity.append(R['solidity'])
        extent.append(R['extent'])
    
    return [s.median(area),s.median(eccentricity),s.median(orientation),s.median(bbox_area),s.median(convex_area),s.median(solidity),s.median(extent)]

    

# Testing and Training on Combined Data

In [15]:
region_features=[]
classes=[]
test_region = []
train_region = []
train_classes_region = []
test_classes_region =[]

for root, dirs, files in os.walk('Data/Combined_Data'):
    for filename in files:
        cropped_image =  io.imread(root+'/'+filename)
        image_class = root.replace("\\","/").split('/')[2]
        print("Processed Image: "+str(filename)+" Class: "+str(image_class))
        region_features.append(calculate_region_props(morph.label(segment_image_dynamic(cropped_image))))
        classes.append(int(image_class))
        
train_region,test_region,train_classes_region,test_classes_region=train_test_split(region_features, classes, test_size=0.33)

Processed Image: G0166380.JPG Class: 1
Processed Image: G0166381.JPG Class: 1
Processed Image: G0166382.JPG Class: 1
Processed Image: G0166383.JPG Class: 1
Processed Image: G0166384.JPG Class: 1
Processed Image: G0296469.JPG Class: 1
Processed Image: G0296472.JPG Class: 1
Processed Image: G0296473.JPG Class: 1


D:\Anaconda_Installed\lib\site-packages\skimage\morphology\misc.py:212: UserWarning: the min_size argument is deprecated and will be removed in 0.16. Use area_threshold instead.
  "0.16. Use area_threshold instead.")
D:\Anaconda_Installed\lib\site-packages\skimage\measure\_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
D:\Anaconda_Installed\lib\site-packages\skimage\measure\_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_i

Processed Image: G0296474.JPG Class: 1
Processed Image: G0296476.JPG Class: 1
Processed Image: G0296477.JPG Class: 1
Processed Image: G0296478.JPG Class: 1
Processed Image: G0296481.JPG Class: 1
Processed Image: G0306486.JPG Class: 1
Processed Image: G0306487.JPG Class: 1
Processed Image: G0306493.JPG Class: 1
Processed Image: G0306495.JPG Class: 1
Processed Image: G0346562.JPG Class: 1
Processed Image: G0346563.JPG Class: 1
Processed Image: G0346565.JPG Class: 1
Processed Image: IMG_4444.JPG Class: 10
Processed Image: IMG_4480.JPG Class: 10
Processed Image: IMG_4488.JPG Class: 10
Processed Image: IMG_4490.JPG Class: 10
Processed Image: IMG_4490_1.JPG Class: 10
Processed Image: IMG_4490_1_2.JPG Class: 10
Processed Image: IMG_4490_1_2_3.JPG Class: 10
Processed Image: IMG_4490_1_2_3_4.JPG Class: 10
Processed Image: IMG_4490_1_2_3_4_5.JPG Class: 10
Processed Image: IMG_4498.JPG Class: 10
Processed Image: IMG_4498_1.JPG Class: 10
Processed Image: IMG_4498_1_2.JPG Class: 10
Processed Image:

Processed Image: G0276459.JPG Class: 3
Processed Image: G0276460.JPG Class: 3
Processed Image: G0276461.JPG Class: 3
Processed Image: G0286462.JPG Class: 3
Processed Image: G0286463.JPG Class: 3
Processed Image: G0286464.JPG Class: 3
Processed Image: G0286465.JPG Class: 3
Processed Image: G0286466.JPG Class: 3
Processed Image: G0286467.JPG Class: 3
Processed Image: G0296469.JPG Class: 3
Processed Image: G0296472.JPG Class: 3
Processed Image: G0296473.JPG Class: 3
Processed Image: G0296474.JPG Class: 3
Processed Image: G0296475.JPG Class: 3
Processed Image: G0296476.JPG Class: 3
Processed Image: G0296477.JPG Class: 3
Processed Image: G0296478.JPG Class: 3
Processed Image: G0296479.JPG Class: 3
Processed Image: G0296480.JPG Class: 3
Processed Image: G0296481.JPG Class: 3
Processed Image: G0296482.JPG Class: 3
Processed Image: G0296483.JPG Class: 3
Processed Image: G0296484.JPG Class: 3
Processed Image: G0306486.JPG Class: 3
Processed Image: G0306487.JPG Class: 3
Processed Image: G0306488

Processed Image: IMG_4428.JPG Class: 4
Processed Image: IMG_4428_1.JPG Class: 4
Processed Image: IMG_4428_1_2.JPG Class: 4
Processed Image: IMG_4428_1_2_3.JPG Class: 4
Processed Image: IMG_4428_1_2_3_4.JPG Class: 4
Processed Image: IMG_4429.JPG Class: 4
Processed Image: IMG_4430.JPG Class: 4
Processed Image: IMG_4438.JPG Class: 4
Processed Image: IMG_4442.JPG Class: 4
Processed Image: IMG_4448.JPG Class: 4
Processed Image: IMG_4450.JPG Class: 4
Processed Image: IMG_4452.JPG Class: 4
Processed Image: IMG_4456.JPG Class: 4
Processed Image: IMG_4456_1.JPG Class: 4
Processed Image: IMG_4456_1_2.JPG Class: 4
Processed Image: IMG_4456_1_2_3.JPG Class: 4
Processed Image: IMG_4456_1_2_3_4.JPG Class: 4
Processed Image: IMG_4456_1_2_3_4_5.JPG Class: 4
Processed Image: IMG_4457.JPG Class: 4
Processed Image: IMG_4464.JPG Class: 4
Processed Image: IMG_4484.JPG Class: 4
Processed Image: IMG_4484_1.JPG Class: 4
Processed Image: IMG_4484_1_2.JPG Class: 4
Processed Image: IMG_4484_1_2_3.JPG Class: 4
Pr

# Classification

In [17]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(train_region,train_classes_region)
predicted_classes = model.predict(test_region)
print("\nKNN Confusion Matrix")
print(confusion_matrix(test_classes_region, predicted_classes))
print("KNN Accuracy Score")
print(accuracy_score(test_classes_region,predicted_classes))


KNN Confusion Matrix
[[ 0  3  2  1  0  0  0  1  0  0  0]
 [ 0 23 11  6  6  0  1  1  0  0  0]
 [ 3 12 16 11  6  0  1  1  0  0  0]
 [ 0 15 12  6  5  0  0  1  0  0  0]
 [ 0  4  7 11  6  1  1  0  0  0  0]
 [ 0  3  1  3  2  0  0  0  0  0  0]
 [ 0  1  0  3  0  0  0  0  0  0  0]
 [ 1  1  2  1  0  0  0  1  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0]
 [ 0  1  1  1  5  0  2  0  0  0  0]
 [ 0  1  3  0  0  0  0  0  0  0  0]]
KNN Accuracy Score
0.25


# Test

In [175]:
# for root, dirs, files in os.walk('Data/Combined_Data'):
#     for filename in files:
#         print("Processed Image: "+str(filename))
#         cropped_image =  io.imread(root+'/'+filename)
#         plt.figure()
#         plt.imshow(cropped_image)
#         plt.show()
#         segment_image(cropped_image)